In [6]:
%load_ext autoreload
%autoreload 2

sys.path.insert(0, '..')
import pyseaflux as sf

In [8]:
ethz = xr.open_dataset('/home/gregorl/updata/pco2_oceansoda-ethz/OS-ETHZ-GRaCER-v2021a_1982-2020.nc')

In [40]:
%time k = sf.fco2_pco2_conversion.virial_coeff(ethz.temperature + 273.15, 1, checks=True)

CPU times: user 1.73 s, sys: 524 ms, total: 2.25 s
Wall time: 2.25 s


In [39]:
%time k = sf.fco2_pco2_conversion.virial_coeff(ethz.temperature + 273.15, 1, checks=False)

CPU times: user 1.57 s, sys: 405 ms, total: 1.98 s
Wall time: 1.98 s


In [42]:
%time k = sf.auxiliary_equations.temperature_correction(ethz.temperature, 25)

CPU times: user 185 ms, sys: 188 ms, total: 373 ms
Wall time: 371 ms


In [53]:
%time k = sf.pCO2_to_fCO2(ethz.spco2, ethz.temperature, checks=False)

CPU times: user 2.04 s, sys: 875 ms, total: 2.92 s
Wall time: 2.91 s


In [49]:
%time k = sf.pCO2_to_fCO2(ethz.spco2, ethz.temperature, checks=True)

CPU times: user 2.33 s, sys: 889 ms, total: 3.21 s
Wall time: 3.21 s


In [69]:
%time k = sf.fCO2_to_pCO2(ethz.sfco2, ethz.temperature, checks=False)

CPU times: user 1.79 s, sys: 797 ms, total: 2.58 s
Wall time: 2.58 s


In [136]:
%time r = sf.solubility.solubility_weiss1974(ethz.salinity, ethz.temperature + 273.15, checks=True)

CPU times: user 1.09 s, sys: 5.68 s, total: 6.77 s
Wall time: 6.81 s


In [101]:
wind = xr.open_dataset(
    '/home/gregorl/Projects/2021-SeaFlux/data/processed/wind_products_mon1deg_moments_1982-2020.nc'
).ERA5[1].assign_coords(time=lambda x: x.time.values.astype('datetime64[M]'))

atm = xr.open_dataarray(
    '/home/gregorl/Projects/2021-SeaFlux/data/SeaFlux_v2021.04/SeaFlux_v2021.04_pco2atm_1982-2020.nc'
).assign_coords(time=lambda x: x.time.values.astype('datetime64[M]'))

pres = xr.open_dataarray(
    '/home/gregorl/Projects/2021-SeaFlux/data/processed/era5_mslp_monthly.nc'
).assign_coords(time=lambda x: x.time.values.astype('datetime64[M]'))

In [100]:
kw = sf.gas_transfer_velocity.k_Ho06(wind, ethz.temperature)

In [122]:
flux = sf.flux_bulk(ethz.temperature, ethz.temperature, ethz.spco2, atm, pres / 100, kw)

In [124]:
integrated = (flux * area * 365).sum(['lat', 'lon'])

In [126]:
integrated.resample(time='1AS').mean()

<xarray.Dataset>
Dimensions:       (time: 39)
Coordinates:
  * time          (time) datetime64[ns] 1982-01-01 1983-01-01 ... 2020-01-01
    moment        int64 2
Data variables:
    fgco2         (time) float64 -1.71e+15 -1.72e+15 ... -2.88e+15 -2.757e+15
    area          (time) float64 1.81e+27 1.81e+27 ... 1.81e+27 1.81e+27
    fgco2_global  (time) float64 -3.183e+32 -3.202e+32 ... -5.361e+32 -5.133e+32